# Deep Crossing

该模型使用 Field 的概念，将每一个特征使用 Embedding 嵌入到低维度空间内。如果有 $n$ 个特征，那么需要把每一个维度 Embedding 后的结果拼接起来，最后送入残差网络，进行优化，得到最终的结果。

In [1]:
# build train data
import os
import numpy as np

BASEDIR = os.getcwd()

fields_dict = {}
lines = None

with open(os.path.join(BASEDIR, 'assets/datasets/criteo_ctr/small_train.txt')) as f:
    lines = f.readlines()

fields_dict = {}
for line in lines:
    line = line.strip('\n')

    for elem in line.split(' ')[1:]:
        field, feature, _ = elem.split(':')

        if field not in fields_dict:
            fields_dict[field] = {'index': len(fields_dict), 'features': {}, 'last_idx': -1}

        if feature not in fields_dict[field]['features']:
            fields_dict[field]['features'][feature] = fields_dict[field]['last_idx'] + 1
            fields_dict[field]['last_idx'] = fields_dict[field]['last_idx'] + 1

for field in fields_dict.keys():
    if 'none' not in fields_dict[field]['features']:
        fields_dict[field]['features']['none'] = fields_dict[field]['last_idx'] + 1
        fields_dict[field]['last_idx'] = fields_dict[field]['last_idx'] + 1


def init_field_tensor(fields_dict):
    init_tensor = np.zeros((len(fields_dict), 1))
    for field in fields_dict.keys():
        init_tensor[fields_dict[field]['index']] = fields_dict[field]['last_idx']
    return init_tensor.astype(int)


X_train = []
y_train = []

for line in lines:
    line = line.strip('\n')
    elems = line.split(' ')
    y_train.append(float(elems[0]))

    init_tensors = init_field_tensor(fields_dict)
    for elem in elems[1:]:
        field, feature, _ = elem.split(':')
        field_idx = fields_dict[field]['index']
        feature_idx = fields_dict[field]['features'][feature]
        init_tensors[field_idx] = feature_idx
    X_train.append(init_tensors)

X_train = np.concatenate(X_train, 1)
y_train = np.array(y_train)

In [2]:
# build embedding layer

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np


class FieldEmbeddingBlock(nn.Module):
    def __init__(self, fields_dict, embedding_size):
        super(FieldEmbeddingBlock, self).__init__()
        self.fields_dict = fields_dict
        self.fields_embedding = {}
        self.embedding_size = embedding_size

        for field in self.fields_dict.keys():
            field_idx = self.fields_dict[field]['index']
            self.fields_embedding[field_idx] = nn.Embedding(len(self.fields_dict[field]['features']),
                                                            self.embedding_size)

    def forward(self, input_field_tensor):
        out = torch.zeros(input_field_tensor.shape[0] * self.embedding_size, input_field_tensor.shape[1])
        for field in self.fields_dict.keys():
            field_idx = self.fields_dict[field]['index']
            idx = field_idx * self.embedding_size
            out[idx:idx + self.embedding_size, :] = self.fields_embedding[field_idx](input_field_tensor[field_idx, :])
        return out.double()


class ResidualBlock(nn.Module):
    def __init__(self, input_dimension, hidden_dimension):
        super(ResidualBlock, self).__init__()
        self.input_dimension = input_dimension
        self.hidden_dimension = hidden_dimension
        self.h1 = nn.Linear(self.input_dimension, self.hidden_dimension, bias=True).double()
        self.h2 = nn.Linear(self.hidden_dimension, self.input_dimension, bias=True).double()

    def forward(self, x):
        residual = x
        out = F.relu(self.h1(x))
        out = self.h2(out)
        out = F.relu(out + residual)
        return out


class DeepCrossing(nn.Module):
    def __init__(self, fields_dict, embedding_size, hidden_dimension):
        super(DeepCrossing, self).__init__()

        self.FieldEmbedding = FieldEmbeddingBlock(fields_dict, embedding_size)
        self.res_input_dim = len(fields_dict) * embedding_size
        self.res1 = ResidualBlock(self.res_input_dim, hidden_dimension)
        self.res2 = ResidualBlock(self.res_input_dim, hidden_dimension)
        self.res3 = ResidualBlock(self.res_input_dim, hidden_dimension)
        self.fc = nn.Linear(self.res_input_dim, 1).double()

    def forward(self, x):
        out = self.FieldEmbedding(x)
        out = out.T
        out = self.res1(out)
        out = self.res2(out)
        out = self.res3(out)
        out = F.sigmoid(self.fc(out))
        return out


In [3]:
# PyTorch Version

import torch.optim as optim

device = torch.device('cpu')
LEARNING_RATE = 1e-3

EPOCH = 100
PRINT_STEP = EPOCH / 10
N = len(y_train)

HIDDEN_DIMENSION = 8
EMBEDDING_SIZE = 8
deepCrossing = DeepCrossing(fields_dict, EMBEDDING_SIZE, HIDDEN_DIMENSION)

BATCH_SIZE = 8
loss_fn = nn.BCELoss()
optimizer = optim.Adam(deepCrossing.parameters(), lr=LEARNING_RATE)

for epoch in range(EPOCH):

    index = np.random.randint(0, X_train.shape[0], size=BATCH_SIZE)
    X_batch = torch.from_numpy(X_train[:, index]).long()
    y_batch = torch.from_numpy(y_train[index]).reshape(-1, BATCH_SIZE)

    y_hat = deepCrossing(X_batch).reshape(-1, BATCH_SIZE)
    loss = loss_fn(y_hat, y_batch)

    loss.backward()
    optimizer.step()

    if epoch % PRINT_STEP == 0:
        print('EPOCH: %d, loss: %f' % (epoch, loss))

EPOCH: 0, loss: 0.730989
EPOCH: 10, loss: 0.444881
EPOCH: 20, loss: 0.028902
EPOCH: 30, loss: 0.679103
EPOCH: 40, loss: 0.842047
EPOCH: 50, loss: 0.610865
EPOCH: 60, loss: 0.220920
EPOCH: 70, loss: 0.426588
EPOCH: 80, loss: 0.543801
EPOCH: 90, loss: 0.714459
